In [1]:
B= BraidGroup(3,'a1,a2')
a1 = B([1])
a2 = B([2])
a3 = B([2,1,-2])


In [2]:
import numpy as np

class NP_3braid:
    """Class representing a 3-braid in Xu's NP (or L^(-1)R) form.
    N_start and P_start are the first letters of N and P, resp. (in the a1,a2,a3 alphabet).
    N_syl and P_syl are lists of positive integers corresponding to the lengths of syllables of the NP braid."""
    B = BraidGroup(3)
    def __init__(self, N_start, N_syl, P_start,P_syl):
        self.N_syl = N_syl
        self.P_syl = P_syl
        self.N_start = N_start
        self.P_start = P_start
        self.braid_list = []
        self.band_word = []
        counter = N_start
        for syl in N_syl:
            if counter == 3:
                self.braid_list = self.braid_list + syl*[2,-1,-2]
                self.band_word = self.band_word + syl*[-3]
            else:
                self.braid_list = self.braid_list + syl*[-counter]
                self.band_word = self.band_word + syl*[-counter]
            if counter == 1:
                counter = 3
            else:
                counter = counter-1
        counter = P_start
        for syl in P_syl:
            if counter == 3:
                self.braid_list = self.braid_list + syl*[2,1,-2]
                self.band_word = self.band_word + syl*[3]
            else:
                self.braid_list = self.braid_list + syl*[counter]
                self.band_word = self.band_word + syl*[counter]
            counter = counter%3+1
    def braid(self):
        return B(self.braid_list)
    def length(self):
        return sum(self.N_syl) + sum(self.P_syl)
    def index(self):
        return 3
    def chi(self):
        return self.index() -self.length()
    def exp_sum(self):
        return self.braid.exponent_sum()
    def sl(self):
        return self.exp_sum() - self.index()
    def defect3(self):
        return (-self.chi() - self.sl())/2
    def link(self):
        return Link(self.braid())
    def AP(self):
        return self.link().alexander_polynomial()
    def JP(self):
        return self.link().jones_polynomial()
    def genus(self):
        return (self.chi() + self.braid().components_in_closure()-2)/(-2)
    def sig(self):
        return self.link().signature()
    def components(self):
        return self.link().number_of_components()
    def is_knot(self):
        return self.link().is_knot()
    def nullity(self):
        return len([z for z in (self.link().seifert_matrix() + self.link().seifert_matrix().transpose()).eigenvalues() if z == 0])
    def det(self):
        return (self.link().seifert_matrix() + self.link().seifert_matrix().transpose()).determinant()
    
    def P_syllables(self):
        P = [i for i in self.band_word if i > 0]
        P_syl = []
        current_band = 0
        current_syl = []
        syl_counter = -1
        for i in range(len(P)):
            new_band = P[i]
            if new_band == current_band:
                current_syl.append([new_band,i])
                current_band = new_band
            else:
                if current_syl != []:
                    P_syl.append([current_syl,syl_counter])
                current_syl = [[new_band,i]]
                syl_counter+=1
                current_band = new_band
            if i == len(P)-1:
                P_syl.append([current_syl,syl_counter])
        return P_syl
                
    def N_syllables(self):
        N = [i for i in self.band_word if i < 0]
        N_syl = []
        current_band = 0
        current_syl = []
        syl_counter = -1
        for i in range(len(N)):
            new_band = N[i]
            if new_band == current_band:
                current_syl.append([new_band,i])
                current_band = new_band
            else:
                if current_syl != []:
                    N_syl.append([current_syl,syl_counter])
                current_syl = [[new_band,i]]
                syl_counter+=1
                current_band = new_band
            if i == len(N)-1:
                N_syl.append([current_syl,syl_counter])
        return N_syl
    
    def violates_del4is0(self):
        return (sum(self.P_syl) < 4 -2*self.components() + self.nullity() + abs(self.sig()) + sum(self.N_syl))
    def seifert_form(self):
        #Computes Seifert form of an NP braid with at least two positive syllables.
        P_syl = self.P_syllables()
        P_bands = len([i for i in self.band_word if i > 0])
        N_syl = self.N_syllables() 
        N_bands = len([i for i in self.band_word if i < 0])
        SF = matrix(ZZ,N_bands + P_bands -1, N_bands + P_bands-1,0)
        found = False
        
        curves = N_syl + P_syl
        
        
        for i in range(len(curves)):
            current_syl = curves[i][0]
            current_syl_type = current_syl[0][0]
            look_ahead = 1
            next_syl_type = 0
            while i + look_ahead < len(curves) and abs(current_syl_type) != abs(next_syl_type):
                next_syl_type = curves[i + look_ahead][0][0][0]
                look_ahead+=1
            if abs(current_syl_type) == abs(next_syl_type):
                for j in range(len(current_syl)):
                    if j < len(current_syl)-1:
                        if current_syl_type < 0:
                            curves[i][0][j][0] = abs(current_syl_type)*100 + 11
                            SF[0,curves[i][0][j][1]+1] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                            SF[curves[i][0][j][1]+1,0] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                        else:
                            curves[i][0][j][0] = abs(current_syl_type)*100
                            SF[0,N_bands+curves[i][0][j][1]+1] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                            SF[N_bands+curves[i][0][j][1]+1,0] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                    else:
                        if current_syl_type == next_syl_type:
                            if current_syl_type < 0:
                                curves[i][0][j][0] = abs(current_syl_type)*100 + 11
                                SF[0,curves[i][0][j][1]+1] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                                SF[curves[i][0][j][1]+1,0] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                            else:
                                curves[i][0][j][0] = abs(current_syl_type)*100
                                SF[0,N_bands+curves[i][0][j][1]+1] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                                SF[N_bands+curves[i][0][j][1]+1,0] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                        else:
                            curves[i][0][j][0] = abs(current_syl_type)*100 + 10
                            SF[0,curves[i][0][j][1]+1] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                            SF[curves[i][0][j][1]+1,0] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
            else:
                if not found:
                    for j in range(len(current_syl)):
                        if j < len(current_syl)-1:
                            curves[i][0][j][0] = abs(current_syl_type)*100
                            SF[0,N_bands+curves[i][0][j][1]+1] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                            SF[N_bands+curves[i][0][j][1]+1,0] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                        else:
                            curves[i][0][j][0] = abs(current_syl_type)*100 + 1
                            SF[0,N_bands+curves[i][0][j][1]+1] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                            SF[N_bands+curves[i][0][j][1]+1,0] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                    found = True
                else:
                    for j in range(len(current_syl)):
                        if j<len(current_syl) -1:
                            curves[i][0][j][0] = abs(current_syl_type)*100
                            SF[0,N_bands+curves[i][0][j][1]+1] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                            SF[N_bands+curves[i][0][j][1]+1,0] = int(str(curves[i][0][j][0]) + str(curves[i][0][j][1]))
                        else:
                            curves[i][0][j][0] = abs(current_syl_type)*100 + 20
                            if i < len(curves) -1:
                                k = i+1
                                while k <= len(curves)-1:
                                    for j in range(len(curves[k][0])):
                                        curves[k][0][j][1] = curves[k][0][j][1] -1
                                    k+=1
        
        num_syls = len(curves)
        
        choices ={
            (311,311): -1, (311, 211):-1,(311, 111):1,(311,310):-1,(311,210):-1,(311,110):1,
            (211,311):1,(211,211):-1,(211,111):1,(211,310):1,(211,210):-1,(211,110):1,
            (111,311):-1,(111,211):-1,(111,111):-1,(111,310):-1,(111,210):-1,(111,110):-1,
            (310,210):-1,(310,110):1,(310,300):1,(310,200):-1,(310,100):1,
            (210,310):1,(210,110):1,(210,300):1,(210,200):1,(210,100):1,
            (110,310):-1,(110,210):-1,(110,300):-1,(110,200):-1,(110,100):1,
            (300,300):1,(300,200):-1,(300,100):1,
            (200,300):1,(200,200):1,(200,100):1,
            (100,300):-1,(100,200):-1,(100,100):1,
            (201,300):-1,(300,301): 1,(200,201):1,(301,200):-1,(201,100):1,(301,100):-1,(300,201):-2,
            (310,201):-2,(200,301):-1,(210,301):-1,(100,201):1,(110,201):1,(100,301):-2,(110,301):-2,
            (310,301):1, (210,201):1
        }
        
        #iterate through syllables
        for i in range(num_syls):
            #choose current syllable
            current_syl = curves[i][0]
            current_syl_len = len(current_syl)
            #iterate through the curves of the current syllable
            for j in range(current_syl_len):
                #set current curve, its type (1,2,3), and its sign (0, 1, 10, 11, or 20("empty" curve))
                current_curve = current_syl[j][0]
                current_curve_type = int(str(current_curve)[0])
                current_curve_sign = int(str(current_curve)[1:])
                #Curve begins in N; index appropriately
                if current_curve_sign ==10 or current_curve_sign == 11:
                    current_curve_index = current_syl[j][1]+1
                #curve begins in P; index appropriately
                elif current_curve_sign == 1 or current_curve_sign == 0:
                    current_curve_index = current_syl[j][1] + N_bands+1
                #curve is "empty"; don't bother
                else:
                    continue
                #Assign diagonal entries
                if current_curve_sign == 11:
                    SF[current_curve_index,current_curve_index] = 2
                elif current_curve_sign == 1:
                    SF[current_curve_index,current_curve_index] = -4
                elif current_curve_sign == 0:
                    SF[current_curve_index,current_curve_index] = -2
                else:
                    SF[current_curve_index,current_curve_index] = 0
                #Handle case when curve is not the last in a syllable; i.e., a curve between two syllables
                if j < current_syl_len-1:
                    #The next curve may be empty; need to avoid
                    if int(str(current_syl[j+1][0])[1:])!= 20:
                        if current_curve_sign == 11:
                            SF[current_curve_index,current_curve_index+1] = -1
                            SF[current_curve_index+1,current_curve_index] = -1
                        else:
                            SF[current_curve_index,current_curve_index+1] = 1
                            SF[current_curve_index+1,current_curve_index] = 1
                    continue
                else:
                    #We are at the end of a syllable. See if there is a syllable of the same type to the right.
                    k = 1
                    match = False
                    while (not match) and (i+k < num_syls):
                        match = (current_curve_type == int(str(curves[i+k][0][0][0])[0]))
                        k+=1
                    
                    if match:
                        #find next syllables: all syllables up to and including the matching syllable
                        next_syls = []
                        for l in range(1,k):
                            syl_type = int(str(curves[i+l][0][0][0])[0])
                            next_syls.append((curves[i+l][0],syl_type))
                        
                        if current_curve_sign == 11:
                            #The match must have been a negative syllable, so all intermediary syllables are
                            #also negative
                            for syl in next_syls:
                                next_syl_type = syl[1]
                                if next_syl_type!=current_curve_type:
                                    #Link with final curve of next syllable
                                    next_curve = syl[0][len(syl[0])-1][0]
                                    next_curve_index = syl[0][len(syl[0])-1][1] +1
                                    SF[current_curve_index,next_curve_index] = choices[(current_curve,next_curve)]
                                    SF[next_curve_index,current_curve_index] = choices[(current_curve,next_curve)]
                                else:
                                    #link with first curve of next syllable
                                    next_curve = syl[0][0][0]
                                    next_curve_index = syl[0][0][1]+1
                                    SF[current_curve_index,next_curve_index] = choices[(current_curve,next_curve)]
                                    SF[next_curve_index,current_curve_index] = choices[(current_curve,next_curve)]
                                
                        elif current_curve_sign == 0:
                            for syl in next_syls:
                                next_syl_type = syl[1]
                                if next_syl_type!=current_curve_type:
                                    #Link with last band in next syllable
                                    next_curve = syl[0][len(syl[0])-1][0]
                                    next_curve_index = syl[0][len(syl[0])-1][1] +1 + N_bands
                                    next_curve_sign = int(str(next_curve)[1:])
                                    #Don't do anything if the match is empty
                                    if next_curve_sign == 20:
                                        continue
                                    else:
                                        if next_curve == 201:
                                            if current_curve_type == 3:
                                                look_ahead_syl = next_syls[next_syls.index(syl)+1][0]
                                                if len(look_ahead_syl) > 1:
                                                    SF[current_curve_index,next_curve_index] = -1
                                                    SF[next_curve_index,current_curve_index] = -1
                                                    continue
                                                else:
                                                    SF[current_curve_index,next_curve_index] = -2
                                                    SF[next_curve_index,current_curve_index] = -2
                                                    continue
                                                    
                                            if current_curve_type == 1:
                                                look_ahead_syl = next_syls[next_syls.index(syl)+2][0]
                                                if len(look_ahead_syl)>1:
                                                    SF[current_curve_index,next_curve_index] = 0
                                                    SF[next_curve_index,current_curve_index] = 0
                                                    continue
                                                else:
                                                    SF[current_curve_index,next_curve_index] = 1
                                                    SF[next_curve_index,current_curve_index] = 1
                                                    continue
                                                    
                                        if next_curve == 301:
                                            if current_curve_type == 1:
                                                look_ahead_syl = next_syls[next_syls.index(syl)+1][0]
                                                if len(look_ahead_syl) > 1:
                                                    SF[current_curve_index,next_curve_index] = -1
                                                    SF[next_curve_index,current_curve_index] = -1
                                                    continue
                                                else:
                                                    SF[current_curve_index,next_curve_index] = -2
                                                    SF[next_curve_index,current_curve_index] = -2
                                                    continue
                                                    
                                            if current_curve_type == 2:
                                                look_ahead_syl = next_syls[next_syls.index(syl)+2][0]
                                                if len(look_ahead_syl)>1:
                                                    SF[current_curve_index,next_curve_index] = 0
                                                    SF[next_curve_index,current_curve_index] = 0
                                                    continue
                                                else:
                                                    SF[current_curve_index,next_curve_index] = -1
                                                    SF[next_curve_index,current_curve_index] = -1
                                                    continue
                                        SF[current_curve_index,next_curve_index] = choices[(current_curve,next_curve)]
                                        SF[next_curve_index,current_curve_index] = choices[(current_curve,next_curve)]
                                else:
                                    #Link with first band in next syllable
                                    next_curve = syl[0][0][0]
                                    next_curve_index = syl[0][0][1]+1 + N_bands
                                    next_curve_sign = int(str(next_curve)[1:])
                                    #Don't do anything if the match is empty
                                    if next_curve_sign == 20:
                                        continue
                                    else:
                                        SF[current_curve_index,next_curve_index] = choices[(current_curve,next_curve)]
                                        SF[next_curve_index,current_curve_index] = choices[(current_curve,next_curve)]
                                
                            
                        else:
                            #Need to handle the case when next_syls contains two of the same type.
                            visited = []
                            #Iterate backwards through next_syls so that when a syllable type is duplicated,
                            #the rightmost duplicate is handled.
                            next_syls.reverse()
                            for syl in next_syls:
                                next_syl_type = syl[1]
                                if next_syl_type in visited:
                                    continue
                                visited.append(next_syl_type)
                                if next_syl_type != current_curve_type:
                                    #Different type, link with final curve in syllable
                                    next_curve = syl[0][len(syl[0])-1][0]
                                    next_curve_sign = int(str(next_curve)[1:])
                                    #Set appropriate index
                                    if next_curve_sign <10:
                                        next_curve_index = syl[0][len(syl[0])-1][1] +1 + N_bands
                                    else:
                                        next_curve_index = syl[0][len(syl[0])-1][1] +1
                                    if next_curve_sign == 20:
                                        continue
                                    else:
                                        if next_curve == 201:
                                            if current_curve_type == 3:
                                                look_ahead_syl = next_syls[next_syls.index(syl)-1][0]
                                                if len(look_ahead_syl) > 1:
                                                    SF[current_curve_index,next_curve_index] = -1
                                                    SF[next_curve_index,current_curve_index] = -1
                                                    continue
                                                else:
                                                    SF[current_curve_index,next_curve_index] = -2
                                                    SF[next_curve_index,current_curve_index] = -2
                                                    continue
                                                    
                                            if current_curve_type == 1:
                                                look_ahead_syl = next_syls[next_syls.index(syl)-2][0]
                                                if len(look_ahead_syl)>1:
                                                    SF[current_curve_index,next_curve_index] = 0
                                                    SF[next_curve_index,current_curve_index] = 0
                                                    continue
                                                else:
                                                    SF[current_curve_index,next_curve_index] = 1
                                                    SF[next_curve_index,current_curve_index] = 1
                                                    continue
                                                    
                                        if next_curve == 301:
                                            if current_curve_type == 1:
                                                look_ahead_syl = next_syls[next_syls.index(syl)-1][0]
                                                if len(look_ahead_syl) > 1:
                                                    SF[current_curve_index,next_curve_index] = -1
                                                    SF[next_curve_index,current_curve_index] = -1
                                                    continue
                                                else:
                                                    SF[current_curve_index,next_curve_index] = -2
                                                    SF[next_curve_index,current_curve_index] = -2
                                                    continue
                                                    
                                            if current_curve_type == 2:
                                                look_ahead_syl = next_syls[next_syls.index(syl)-2][0]
                                                if len(look_ahead_syl)>1:
                                                    SF[current_curve_index,next_curve_index] = 0
                                                    SF[next_curve_index,current_curve_index] = 0
                                                    continue
                                                else:
                                                    SF[current_curve_index,next_curve_index] = -1
                                                    SF[next_curve_index,current_curve_index] = -1
                                                    continue
                                        SF[current_curve_index,next_curve_index] = choices[(current_curve,next_curve)]
                                        SF[next_curve_index,current_curve_index] = choices[(current_curve,next_curve)]
                                else:
                                    #same type, link with first curve in syllable
                                    next_curve = syl[0][0][0]
                                    next_curve_index = syl[0][0][1]+1 + N_bands
                                    next_curve_sign = int(str(next_curve)[1:])
                                    if next_curve_sign == 20:
                                        continue
                                    else:
                                        SF[current_curve_index,next_curve_index] = choices[(current_curve,next_curve)]
                                        SF[next_curve_index,current_curve_index] = choices[(current_curve,next_curve)]
                            
                    else:
                        #No match was found; must be in one of the last three syllables
                        next_syls = []
                        #In the worst case, i + k = num_syls, so this will take us to the end of the braid
                        for l in range(1,k):
                            syl_type = int(str(curves[i+l][0][0][0])[0])
                            next_syls.append((curves[i+l][0],syl_type))
                        for syl in next_syls:
                            #Indexing is different when we're not looking at the sign 1 curve, which always occurs on the 
                            #third to final index
                            if i != num_syls-3:
                                next_curve = syl[0][len(syl[0])-1][0]
                                next_curve_index = syl[0][len(syl[0])-1][1] +1 + N_bands
                                next_curve_sign = int(str(next_curve)[1:])
                                if next_curve_sign == 20:
                                    continue
                                else:
                                    SF[current_curve_index,next_curve_index] = choices[(current_curve,next_curve)]
                                    SF[next_curve_index,current_curve_index] = choices[(current_curve,next_curve)]
                            else:
                                #The sign 1 curve requires some special indexing; its next syllables invovle "empty" curves
                                if len(syl[0])-2 >=0:
                                    next_curve = syl[0][len(syl[0])-2][0]
                                    next_curve_index = syl[0][len(syl[0])-2][1] + 1 + N_bands
                                    next_curve_sign = int(str(next_curve)[1:])
                                else:
                                    continue
                                if next_curve_sign == 20:
                                    continue
                                else:
                                    SF[current_curve_index,next_curve_index] = choices[(current_curve,next_curve)]
                                    SF[next_curve_index,current_curve_index] = choices[(current_curve,next_curve)]
                    
            
                
            
        return matrix(SF)
                
                
                
                
                
                

In [8]:
b = NP_3braid(3,[3],2,[1,1,1,1,1,1])
print b.violates_del4is0()
print "Components = " + str(b.components())
print "Signature = " + str(b.sig())
print "Nullity = " + str(b.nullity())
print "Determinant = " + str(b.det())
print "Num negative bands = " + str(sum(b.N_syl))
print "Num positive bands = " + str(sum(b.P_syl))
print "Negative bands + 2 - 2r + nullity + abs(sig) = " + str(sum(b.N_syl) + 4 - 2*b.components() + b.nullity() + abs(b.sig()))
print "Difference = " + str(sum(b.P_syl) - (sum(b.N_syl) + 4 + abs(b.sig()) - 2*b.components() + b.nullity()))
print b.sig() - (sum(b.N_syl)-sum(b.P_syl))

False
Components = 2
Signature = 0
Nullity = 1
Determinant = 0
Num negative bands = 3
Num positive bands = 6
Negative bands + 2 - 2r + nullity + abs(sig) = 4
Difference = 2
3


In [3]:
def N_minus_P(n_seq,p_seq):
    p_index = 0
    n_index = 0
    while n_index < len(n_seq) and p_seq[p_index]!=n_seq[n_index]:
        n_index +=1
    while p_index < len(p_seq) and n_index < len(n_seq):
        p_index +=1
        n_index +=1
    if n_index == len(n_seq):
        return p_seq[p_index:],1
    else:
        return n_seq[n_index:],-1

def syllable_error(N,P,p_start):
    if p_start ==3:
        p_start =0
    if len(N)%3==0:
        n_start = 1
    if len(N)%3==1:
        n_start = 0
    if len(N)%3==2:
        n_start = 2
    p_iter = 1
    p_next = (p_start +1)%3
    n_iter = 1
    n_next = (n_start+1)%3
    p_seq = [p_start]
    n_seq = [n_start]
    while p_iter < len(P):
        p_seq.append(p_next)
        p_next = (p_next+1)%3
        p_iter += 1
    while n_iter < len(N):
        n_seq.append(n_next)
        n_next = (n_next+1)%3
        n_iter += 1
    diff,sign = N_minus_P(n_seq,p_seq)
    epsilon = sign*len([i for i in diff if i==0])
    return epsilon

In [4]:
from random import choice,randint

def gen_random_braid(n_len,p_len):
    if n_len%3==0:
        if p_len%3==0:
            p_start = choice([2,3])
        elif p_len%3==1:
            p_start = 2
        else:
            p_start = 3
    if n_len%3==1:
        if p_len%3==0:
            p_start = 2
        elif p_len%3==1:
            p_start = choice([1,2])
        else:
            p_start = 1
    if n_len%3==2:
        if p_len%3==0:
            p_start = 3
        elif p_len%3==1:
            p_start = 1
        else:
            p_start = choice([1,3])
    return [randint(1,3) for _ in range(n_len)],[randint(1,3) for _ in range(p_len)],p_start
        

In [ ]:
#Tests signature conjecture on randomly generated braids.
#Braids are chosen at random from those satisfying the following conditions:
# 1) syllables have maximum length 3
# 2) N and P have maximum length 10

from random import choice,randint
import time

start_time = time.clock()

conj_true = 0
conj_false = []
has_nullity = []
n_num_syl = 1
counter = 0
max_iters = 50000

while counter < max_iters:
    p_len = randint(1,10)
    n_len = randint(1,10)
    N,P,p_start = gen_random_braid(n_len,p_len)
    b = NP_3braid(3,N,p_start,P)
    if b.nullity() > 0:
        has_nullity.append(b)
        print "Found braid with nullity!"
        print b.band_word
    if len(P) >=len(N):
        if b.sig() == sum(b.N_syl) - sum(b.P_syl) + 2*syllable_error(N,P,p_start) - b.nullity():
            conj_true+=1
            #print b.band_word
        else:
            conj_false.append(b)
            print "Found counterexample!"
            print b.band_word
    else:
        if b.sig() == sum(b.N_syl) - sum(b.P_syl) + 2*syllable_error(N,P,p_start) + b.nullity():
            conj_true+=1
            #print b.band_word
        else:
            conj_false.append(b)
            print "Found counterexample!"
            print b.band_word
    counter+=1
    if counter%100==0:
        print str(counter) + " " + str(time.clock() - start_time)

100 63.591859
200 147.246284
300 224.123464
400 304.092126
500 369.296297
600 459.891724
700 545.521523
800 623.246983
900 696.860877
1000 760.795215
1100 854.651969
1200 930.669531
1300 1011.083868
1400 1083.942591
1500 1157.909665
1600 1235.453541
1700 1316.458972
1800 1391.327652
1900 1462.536843
2000 1552.536258
2100 1622.431073
2200 1686.738059
2300 1752.167821
2400 1824.801408
2500 1911.522503
2600 1994.655118
2700 2064.276309
2800 2139.338472
2900 2217.130553
3000 2294.750729
3100 2358.003227
3200 2442.163952
3300 2501.933249
3400 2584.689781
3500 2647.289009
3600 2720.210777
3700 2791.020107
3800 2864.082998
3900 2933.152168
4000 3007.464747
4100 3070.364256
4200 3146.403227
4300 3245.091458
4400 3316.626145
4500 3378.209646
4600 3440.290708
4700 3498.749452
4800 3570.50482
4900 3639.811429
5000 3723.41667
5100 3787.82292
5200 3865.191138
5300 3926.761993
5400 3996.66884
5500 4081.219218
5600 4138.808872
5700 4199.766025
5800 4271.862146
5900 4349.331583
6000 4424.595624
6100 4

In [6]:
def symmetric_row_reduce(N, display = False):
    n = N.ncols()
    M = matrix(SR,1,1,0).block_sum(N[1:,1:].change_ring(SR))
    N = N.change_ring(SR) - M
    
    for i in range(1,n):
        if M[i,i]==0:
            look_ahead =1
            while i+look_ahead < n:
                if M[i+look_ahead,i+look_ahead] != 0:
                    M.swap_rows(i,i+look_ahead)
                    N.swap_rows(i,i+look_ahead)
                    M.swap_columns(i,i+look_ahead)
                    N.swap_columns(i,i+look_ahead)
                    if display:
                        print M.str()
                    break
                else:
                    look_ahead+=1
            if i + look_ahead == n:
                break
        temp = abs(M[i,i])
        M.rescale_row(i,1/sqrt(temp))
        M.rescale_col(i,1/sqrt(temp))
        if display:
            print M.str()
        if M[i,i] > 0:
            for j in range(i+1,n):
                M.add_multiple_of_row(j,i,-M[j,i])
                M.add_multiple_of_column(j,i,-M[i,j])
                if display:
                    print M.str()
        else:
            for j in range(i+1,n):
                M.add_multiple_of_row(j,i,M[j,i])
                M.add_multiple_of_column(j,i,M[i,j])
                if display:
                    print M.str()
                        
    for i in range(n):
        if abs(M[i,i]-1) < 10**(-2):
            M[i,i] = 1
        elif abs(M[i,i]+1) < 10**(-2):
            M[i,i] = -1
        else:
            M[i,i] = 0
        for j in range(i+1,n):
            if abs(M[i,j]) < 10**(-1):
                M[i,j] = 0
                M[j,i] = 0
                
    return M+N
                

In [9]:
#Collects Seifert form data for randomly generated braids.
#Braids are chosen at random from those satisfying the following conditions:
# 1) syllables have maximum length 3
# 2) N and P have maximum syllables 10

from random import choice,randint
import time

start_time = time.clock()

signature_data = []
has_nullity = []
errors = []
n_num_syl = 1
counter = 0
max_iters = 50000

while counter < max_iters:
    p_len = randint(3,10)
    n_len = randint(1,10)
    N,P,p_start = gen_random_braid(n_len,p_len)
    b = NP_3braid(3,N,p_start,P)
    if b.nullity > 0:
        has_nullity.append(b)
    M = b.seifert_form()
    X = symmetric_row_reduce(M)
    if b.sig() != sum(X.diagonal()):
        print "Error in program: miscalculated signature"
        errors.append(b)
    else:
        current_datum = []
        for i in range(1,X.ncols()):
            current_datum.append((X[0,i],X[i,i]))
        signature_data.append(current_datum)
    
    counter+=1
    if counter%100==0:
        print str(counter) + " " + str(time.clock() - start_time)

100 60.491651
200 127.256011
300 195.355557
400 262.129069
500 327.003274
600 388.87052
700 464.826782
800 540.077936
900 600.951099
1000 664.489063
1100 737.234132
1200 805.98543
1300 872.827328
1400 945.309186
1500 1008.864356
1600 1084.481047
1700 1150.637699
1800 1224.356363
1900 1298.862471
2000 1360.628843
2100 1432.838821
2200 1497.862797
2300 1568.953987
2400 1629.113868
2500 1698.171084
2600 1762.11754
2700 1839.994588
2800 1913.059068
2900 1982.22355
3000 2045.853475
3100 2105.758371
3200 2174.068447
3300 2240.089342
3400 2310.772311
3500 2372.271369
3600 2460.71616
3700 2521.312291
3800 2593.86214
3900 2667.638046
4000 2739.698309
4100 2807.762562
4200 2872.182587
4300 2944.88043
4400 3012.730492
4500 3075.66632
4600 3141.6334
4700 3213.646565
4800 3276.689221
4900 3347.13355
5000 3410.028112
5100 3477.680101
5200 3546.146703
5300 3616.985705
5400 3681.620549
5500 3758.668273
5600 3822.952012
5700 3892.533618
5800 3960.136488
5900 4032.631962
6000 4102.001742
6100 4173.41026

In [32]:
data = matrix(ZZ, 34, 3,0)

#LABELING CONVENTION: 
#trailing 0: curve occurs at beginning of syllable
#trailing 1: curve occurs in middle of syllable
#trailing 2: curve occurs at end of syllable.
#trailing 3: curve is unique to its syllable.
data[0,0] = 311*100
data[1,0] = 311*100+1
data[2,0] = 311*100+2
data[3,0] = 211*100
data[4,0] = 211*100+1
data[5,0] = 211*100+2
data[6,0] = 111*100
data[7,0] = 111*100+1
data[8,0] = 111*100+2
data[9,0] = 310*100
data[10,0] = 210*100
data[11,0] = 110*100
data[12,0] = 300*100
data[13,0] = 300*100 + 1
data[14,0] = 300*100 + 2
data[15,0] = 200*100
data[16,0] = 200*100 + 1
data[17,0] = 200*100 + 2
data[18,0] = 100*100
data[19,0] = 100*100 + 1
data[20,0] = 100*100 + 2
data[21,0] = 301*100 + 2
data[22,0] = 311*100 + 3
data[23,0] = 310*100 + 3
data[24,0] = 301*100 + 3
data[25,0] = 300*100 + 3
data[26,0] = 211*100 + 3
data[27,0] = 210*100 + 3
data[28,0] = 200*100 + 3
data[29,0] = 111*100 + 3
data[30,0] = 110*100 + 3
data[31,0] = 100*100 + 3
data[32,0] = 201*100 + 2
data[33,0] = 201*100 + 3

#COLUMNS:
#The zeroth column is the labeling column;
#the first column records the number of 1's
#the last column records the numer of -1's

for datum in signature_data:
    temp = []
    for curve in datum:
        temp.append({
            "Curve":int(str(curve[0])[0:3]),
            "Type":int(str(curve[0])[0]),
            "Index":int(str(curve[0])[3:]),
            "Entry": curve[1]
        })
    new_datum = sorted(temp, key = lambda k:k["Index"])
    for i in range(len(new_datum)):
        current = new_datum[i]
        if i == 0:
            prev = {"Type": None}
        else:
            prev = new_datum[i-1]
        if i == len(new_datum)-1:
            proc = {"Type": None}
        else:
            proc = new_datum[i+1]
        if prev["Type"]!=current["Type"] or proc["Type"] != current["Type"]:
            if prev["Type"] != current["Type"] and proc["Type"] == current["Type"]:
                if current["Entry"] == 1:
                    if current["Curve"] == 311:
                        data[22,1]+=1
                    elif current["Curve"] == 310:
                        data[23,1]+=1
                    elif current["Curve"] == 301:
                        data[24,1]+=1
                    elif current["Curve"] == 300:
                        data[25,1]+=1
                    elif current["Curve"] == 211:
                        data[26,1]+=1
                    elif current["Curve"] == 210:
                        data[27,1]+=1
                    elif current["Curve"] == 200:
                        data[28,1]+=1
                    elif current["Curve"] == 111:
                        data[29,1]+=1
                    elif current["Curve"] == 110:
                        data[30,1]+=1
                    elif current["Curve"] == 201:
                        data[33,1]+=1
                    else:
                        data[31,1]+=1
                else:
                    if current["Curve"] == 311:
                        data[22,2]+=1
                    elif current["Curve"] == 310:
                        data[23,2]+=1
                    elif current["Curve"] == 301:
                        data[24,2]+=1
                    elif current["Curve"] == 300:
                        data[25,2]+=1
                    elif current["Curve"] == 211:
                        data[26,2]+=1
                    elif current["Curve"] == 210:
                        data[27,2]+=1
                    elif current["Curve"] == 200:
                        data[28,2]+=1
                    elif current["Curve"] == 111:
                        data[29,2]+=1
                    elif current["Curve"] == 110:
                        data[30,2]+=1
                    elif current["Curve"] == 201:
                        data[33,2]+=1
                    else:
                        data[31,1]+=1
            elif prev["Type"] != current["Type"]:
                if current["Entry"] == 1:
                    if current["Curve"] == 311:
                        data[0,1]+=1
                    elif current["Curve"] == 310:
                        data[9,1]+=1
                    elif current["Curve"] == 301:
                        data[21,1]+=1
                    elif current["Curve"] == 300:
                        data[12,1]+=1
                    elif current["Curve"] == 211:
                        data[3,1]+=1
                    elif current["Curve"] == 210:
                        data[10,1]+=1
                    elif current["Curve"] == 200:
                        data[15,1]+=1
                    elif current["Curve"] == 111:
                        data[6,1]+=1
                    elif current["Curve"] == 110:
                        data[11,1]+=1
                    else:
                        data[18,1]+=1
                else:
                    if current["Curve"] == 311:
                        data[0,2]+=1
                    elif current["Curve"] == 310:
                        data[9,2]+=1
                    elif current["Curve"] == 301:
                        data[21,2]+=1
                    elif current["Curve"] == 300:
                        data[12,2]+=1
                    elif current["Curve"] == 211:
                        data[3,2]+=1
                    elif current["Curve"] == 210:
                        data[10,2]+=1
                    elif current["Curve"] == 200:
                        data[15,2]+=1
                    elif current["Curve"] == 111:
                        data[6,2]+=1
                    elif current["Curve"] == 110:
                        data[11,2]+=1
                    else:
                        data[18,2]+=1
            else:
                if current["Entry"] == 1:
                    if current["Curve"] == 311:
                        data[2,1]+=1
                    elif current["Curve"] == 310:
                        data[9,1]+=1
                    elif current["Curve"] == 301:
                        data[21,1]+=1
                    elif current["Curve"] == 300:
                        data[14,1]+=1
                    elif current["Curve"] == 211:
                        data[5,1]+=1
                    elif current["Curve"] == 210:
                        data[10,1]+=1
                    elif current["Curve"] == 200:
                        data[17,1]+=1
                    elif current["Curve"] == 111:
                        data[8,1]+=1
                    elif current["Curve"] == 110:
                        data[11,1]+=1
                    elif current["Curve"]==201:
                        data[32,1] +=1
                    else:
                        data[20,1]+=1
                else:
                    if current["Curve"] == 311:
                        data[2,2]+=1
                    elif current["Curve"] == 310:
                        data[9,2]+=1
                    elif current["Curve"] == 301:
                        data[21,2]+=1
                    elif current["Curve"] == 300:
                        data[14,2]+=1
                    elif current["Curve"] == 211:
                        data[5,2]+=1
                    elif current["Curve"] == 210:
                        data[10,2]+=1
                    elif current["Curve"] == 200:
                        data[17,2]+=1
                    elif current["Curve"] == 111:
                        data[8,2]+=1
                    elif current["Curve"] == 110:
                        data[11,2]+=1
                    elif current["Curve"]==201:
                        data[32,2] +=1
                    else:
                        data[20,2]+=1
        else:
            if current["Entry"] == 1:
                if current["Curve"] == 311:
                    data[1,1]+=1
                elif current["Curve"] == 300:
                    data[13,1]+=1
                elif current["Curve"] == 211:
                    data[4,1]+=1
                elif current["Curve"] == 200:
                    data[16,1]+=1
                elif current["Curve"] == 111:
                    data[7,1]+=1
                else:
                    data[19,1]+=1
            else:
                if current["Curve"] == 311:
                    data[1,2]+=1
                elif current["Curve"] == 300:
                    data[13,2]+=1
                elif current["Curve"] == 211:
                    data[4,2]+=1
                elif current["Curve"] == 200:
                    data[16,2]+=1
                elif current["Curve"] == 111:
                    data[7,2]+=1
                else:
                    data[19,2]+=1
        
    